In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import scipy.integrate as itp

plt.rc('font', size=18)
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('legend', fontsize=10)
plt.rc('mathtext', fontset='cm')
plt.rc('text', usetex=True)
#plt.rc('font', family='nimbus sans narrow') 

In [ ]:
def v_f(x, t):
    return np.array((a*x[1], 0.))

def grad_v_f(x, t):
    return np.array(((0., a), (0., 0.)))
def F(y, t):
    state, adjoint = y[:2], y[2:]
    return np.concatenate((-v_r * adjoint / np.linalg.norm(adjoint) + v_f(state, t),
                    -grad_v_f(state, t).transpose() @ adjoint))
def F_obs(y, t):
    state, adjoint = y[:2], y[2:]
    return np.concatenate((-v_r * adjoint / np.linalg.norm(adjoint) + v_f(state, t),
                    -grad_v_f(state, t).transpose() @ adjoint))

In [ ]:
#nx, ny = 101, 101
nx, ny = 31, 31
bounds = np.array(((-0.2, 1.2), (-0.5, 0.5)))
x, y = np.meshgrid(np.linspace(*bounds[0], nx), np.linspace(*bounds[1], ny), indexing='ij')

In [ ]:
def wind_plot(no_wind_norm=False):    
    fig, ax = plt.subplots()
    
    if not no_wind_norm:
        ax.pcolormesh(x, y, np.abs(y), cmap='turbo', shading='gouraud')
    rect = patches.Rectangle((bounds[0, 0], bounds[1, 0]), bounds[0, 1] - bounds[0, 0], bounds[1, 1] - bounds[1, 0], facecolor='white', alpha=0.3)
    ax.add_patch(rect)
    n_quiv = 2
    off = 1
    ax.grid(True, color='black', alpha=0.5)
    ax.quiver(x[off::n_quiv, off::n_quiv], y[off::n_quiv, off::n_quiv], y[off::n_quiv, off::n_quiv], 0 * x[off::n_quiv, off::n_quiv])
    
    ax.scatter(*np.array((0, 0)), color='black', edgecolor='white', s=100, zorder=10)
    ax.scatter(*np.array((1, 0)), color='black', edgecolor='white', s=200, marker='*', zorder=10)
    return fig, ax

In [ ]:
fig, ax = wind_plot()

n_traj, n_t = 8, 60
v_r, a = 1., 1.
trajs = np.zeros((n_traj, n_t, 4))
x0 = np.zeros(2)
angles = np.linspace(0, np.pi/4, n_traj)
for i, angle in enumerate(angles):
    y0 = np.array((0., 0., -np.cos(angle), -np.sin(angle)))
    trajs[i, :] = itp.odeint(F, y0, np.linspace(0, 0.96, n_t))

for i in range(trajs.shape[0]):
    ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=2.5, color='white')
    ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=1.5, color='black' if i != 4 else 'red')
    ax.scatter(trajs[i, -1, 0], trajs[i, -1, 1], color='black', edgecolor='white', marker='D', zorder=10)


ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8 ,1])
ax.set_yticks(np.arange(-0.4, 0.41, 0.2))
ax.set_title(r'\textbf{Linear flow field}')
#plt.savefig('linear_extremals/linear_extremals.pdf')
plt.show()

In [ ]:
fig, ax = wind_plot(no_wind_norm=True)

n_traj, n_t = 100, 60
v_r, a = 1., 1.
trajs = np.zeros((n_traj, n_t, 4))
x0 = np.zeros(2)
angles = np.linspace(0, 2*np.pi, n_traj)
for i, angle in enumerate(angles):
    y0 = np.array((0., 0., -np.cos(angle), -np.sin(angle)))
    trajs[i, :] = itp.odeint(F, y0, np.linspace(0, 0.5, n_t))



for it in range(n_t - 1):
    ax.plot(trajs[:, it, 0], trajs[:, it, 1], linewidth=2.5, alpha=0.2, color='green')

ax.plot(trajs[:, -1, 0], trajs[:, -1, 1], linewidth=2, color='blue')

for i in [2, 6, 10]:
    ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=2.5, color='white')
    ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=1.5, color='black' if i != 4 else 'red')
    ax.scatter(trajs[i, -1, 0], trajs[i, -1, 1], color='black', edgecolor='white', marker='D', zorder=10)

ax.annotate(r'$\mathcal{E}_{\mathbf{x}_0}^{\mathbf{x}}(t)$', (0.4, -0.22), xytext=(0.5, -0.3), color='blue',
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.2', linewidth=0.5),
           arrowprops=dict(edgecolor='blue', arrowstyle='-', linewidth=0.5))

ax.annotate(r'$\mathcal{S}_{\mathbf{x}_0}^{\mathbf{x}}(t)$', (0, -0.2), color='green',
            bbox=dict(facecolor='white', edgecolor='green', boxstyle='round,pad=0.2', linewidth=0.5))

ax.set_xlim(-0.2, 1.2)
ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8 ,1])
ax.set_yticks(np.arange(-0.4, 0.41, 0.2))
#ax.set_title(f'Linear flow field', fontweight="bold")
#plt.savefig('linear_extremals/linear_extremals_set.pdf')
plt.show()

In [ ]:
fig, ax = plt.subplots()
for spine_type in ['top', 'right', 'bottom', 'left']:
    ax.spines[spine_type].set_visible(False)
        
ax.tick_params(axis='both', which='both', length=0)
ax.set_xticklabels([])
ax.set_yticklabels([])

ax.scatter(*np.array((0, 0)), color='black', edgecolor='white', s=100, zorder=10)

n_traj, n_t = 101, 60
v_r, a = 1., 1.
trajs = np.zeros((n_traj, n_t, 4))
x0 = np.zeros(2)
angles = np.linspace(0, 2*np.pi, n_traj)
for i, angle in enumerate(angles):
    y0 = np.array((0., 0., -np.cos(angle), -np.sin(angle)))
    trajs[i, :] = itp.odeint(F, y0, np.linspace(0, 0.5, n_t))
    
ax.plot(trajs[:, -1, 0], trajs[:, -1, 1], linewidth=2, color='blue', alpha=0.5)

step = 10
for i in range(0, n_traj, step):
    ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=2.5, color='white')
    ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=1.5, color='black')
    ax.scatter(trajs[i, -1, 0], trajs[i, -1, 1], color='black', edgecolor='white', marker='D', zorder=10)
    if i < n_traj - 1:
        ax.plot([trajs[i, -1, 0], trajs[i + step, -1, 0]], 
                [trajs[i, -1, 1], trajs[i + step, -1, 1]], linewidth=2, color='red', alpha=0.8)
        step_it = 10
        for it in range(0, n_t, step_it):
            coords = np.stack((trajs[i, it, :2], trajs[i + step, it, :2], trajs[i + step, min(it + step_it, n_t - 1), :2]))
            tri = patches.Polygon(coords, color='orange', alpha=0.7)
            ax.add_patch(tri)
            coords = np.stack((trajs[i, it, :2], trajs[i, min(it + step_it, n_t - 1), :2], trajs[i + step, min(it + step_it, n_t - 1), :2]))
            tri = patches.Polygon(coords, color='green', alpha=0.7)
            ax.add_patch(tri)

ax.set_xlim(-0.6, 0.6)
ax.set_xticks(np.arange(-0.6, 0.61, 0.2))
ax.set_yticks(np.arange(-0.4, 0.41, 0.2))
ax.axis('equal')
#ax.set_title(f'Linear flow field', fontweight="bold")
#plt.savefig('linear_extremals/linear_extremals_set_approx.pdf')
plt.show()

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()


#ax.scatter(*np.array((0, 0)), color='black', edgecolor='white', s=100, zorder=10)

n_traj, n_t = 101, 60
v_r, a = 1., 1.
trajs = np.zeros((n_traj, n_t, 4))
x0 = np.zeros(2)
angles = np.linspace(0, 2*np.pi, n_traj)
times = np.linspace(0, 0.5, n_t)
for i, angle in enumerate(angles):
    y0 = np.array((0., 0., -np.cos(angle), -np.sin(angle)))
    trajs[i, :] = itp.odeint(F, y0, times)

#fig.add_trace(go.Scatter3d(x=trajs[:, -1, 0], y=trajs[:, -1, 1])
#ax.plot(trajs[:, -1, 0], trajs[:, -1, 1], linewidth=2, color='blue', alpha=0.5)

step = 10
for i in range(0, n_traj, step):
    fig.add_trace(go.Scatter3d(x=trajs[i, :, 0], y=trajs[i, :, 1], z=times, mode='lines', line=dict(color='black'), showlegend=False))
    # ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=2.5, color='white')
    # ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=1.5, color='black')
    # ax.scatter(trajs[i, -1, 0], trajs[i, -1, 1], color='black', edgecolor='white', marker='D', zorder=10)
    if i < n_traj - 1:
        # ax.plot([trajs[i, -1, 0], trajs[i + step, -1, 0]], 
        #         [trajs[i, -1, 1], trajs[i + step, -1, 1]], linewidth=2, color='red', alpha=0.8)
        step_it = 10
        for it in range(0, n_t, step_it):
            it_po = min(it + step_it, n_t - 1)
            coords = np.stack((trajs[i, it, :2], trajs[i + step, it, :2], trajs[i + step, it_po, :2]))
            coords_aug = np.stack((np.concatenate((coords[0], [times[it]])), 
                                    np.concatenate((coords[1], [times[it]])), 
                                    np.concatenate((coords[2], [times[it_po]]))))
            fig.add_trace(go.Mesh3d(x=coords_aug.T[0], y=coords_aug.T[1], z=coords_aug.T[2], alphahull=5, opacity=1, color='orange', 
                                    i=np.array([0]), j=np.array([1]), k=np.array([2])))
            # tri = patches.Polygon(coords, color='orange', alpha=0.7)
            # ax.add_patch(tri)
            coords = np.stack((trajs[i, it, :2], trajs[i, it_po, :2], trajs[i + step, it_po, :2]))
            coords_aug = np.stack((np.concatenate((coords[0], [times[it]])), 
                                    np.concatenate((coords[1], [times[it_po]])), 
                                    np.concatenate((coords[2], [times[it_po]]))))
            fig.add_trace(go.Mesh3d(x=coords_aug.T[0], y=coords_aug.T[1], z=coords_aug.T[2], alphahull=5, opacity=1, color='green', 
                        i=np.array([0]), j=np.array([1]), k=np.array([2])))
            # tri = patches.Polygon(coords, color='green', alpha=0.7)
            # ax.add_patch(tri)
git@github.com:bschnitzler/schnitzler-thesis-notebooks.git
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1, y=1, z=0.8/1.5)
)
fig.update_layout(width=800, height=600, scene=dict(zaxis_title='t'), scene_camera=camera)
fig.layout.scene.camera.projection.type = "orthographic"
fig.show()

In [ ]:
import plotly
plotly.io.orca.config.executable = '/home/bastien/anaconda3/envs/sci/bin/orca'
fig.write_image('linear_extremals/linear_extremals_set_approx_3d.png', 
                engine='orca', scale=3)

In [ ]:
from dabry.misc import Utils
Utils.linear_wind_alyt_traj(1, 1, np.array((0, 0)), np.array((1, 0)))

In [ ]:
airspeed = 1
w = 1
def analytic_traj(theta, theta_f):
    x = 0.5 * airspeed / w * (-1 / np.cos(theta_f) * (np.tan(theta_f) - np.tan(theta)) +
                              np.tan(theta) * (1 / np.cos(theta_f) - 1 / np.cos(theta)) -
                              np.log((np.tan(theta_f)
                                      + 1 / np.cos(theta_f)) / (
                                             np.tan(theta) + 1 / np.cos(theta))))
    y = airspeed / w * (1 / np.cos(theta) - 1 / np.cos(theta_f))
    return x, y

In [ ]:
theta_f = 1
points = np.array(list(map(lambda theta: analytic_traj(theta, theta_f), np.linspace(-theta_f*1.1, theta_f, 50))))
plt.plot(*points.T)
theta_f = 0.5
points = np.array(list(map(lambda theta: analytic_traj(theta, theta_f), np.linspace(-theta_f, theta_f, 50))))
plt.plot(*points.T)
x, y = analytic_traj(-theta_f, theta_f)
plt.plot([x, x + np.cos(-theta_f)], [y, y + np.sin(-theta_f)])

theta_f = -0.5 
points = np.array(list(map(lambda theta: analytic_traj(theta, theta_f), np.linspace(theta_f - 0.5*abs(theta_f), theta_f, 50))))
plt.plot(*points.T)
x, y = analytic_traj(theta_f - 0.5*abs(theta_f), theta_f)
plt.plot([x, x + np.cos(theta_f - 0.5*abs(theta_f))], [y, y + np.sin(theta_f - 0.5*abs(theta_f))])

theta_f = 6
points = np.array(list(map(lambda theta: analytic_traj(theta, theta_f), np.linspace(0.99*theta_f, theta_f, 50))))
plt.plot(*points.T)